<a href="https://colab.research.google.com/github/nachofranco17/ProyectoCNN/blob/main/ProyectoCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Probando pushes 3

!pip install kagglehub
import kagglehub
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Descargando chestxray
path = kagglehub.dataset_download("nih-chest-xrays/data")
print("Dataset downloaded to:", path)

csv_path = os.path.join(path, "Data_Entry_2017.csv")
df = pd.read_csv(csv_path)
print(df.head())

In [ ]:
# Viendo algunas imagenes

image_folder = os.path.join(path, "images")
print("Number of images:", len(os.listdir(image_folder)))
print("Example file:", os.listdir(image_folder)[:5])

In [ ]:
img = mpimg.imread(os.path.join(image_folder, os.listdir(image_folder)[0]))
plt.imshow(img, cmap='gray')
plt.axis('off')
plt.show()